In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Load the model
llm = ChatNVIDIA(model="meta/llama-3.1-405b-instruct")
parser = StrOutputParser()

# Initialize history list
history = []

# Prompt to generate the first follow-up question
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "You are a career recommendation bot. You ask questions to understand user interests."),
    ("user", "The user said: {input}. Ask a good follow-up question.")
])

chain1 = prompt1 | llm | parser

# Prompt to generate the second follow-up question based on the previous user answer
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You're a career bot."),
    ("user", "The user responded: {pre}. Ask the next follow-up question to understand their interests further.")
])

chain2 = prompt2 | llm | parser

# Start the interaction
if not history:
    ques1 = "What do you do in your free time?"
    print("\n[Bot]:", ques1)
    ans1 = input("[You]: ")
    history.append(("Q1", ans1))

    # Generate follow-up question (Q2)
    ques2 = chain1.invoke({"input": ans1})
    print("\n[Bot]:", ques2)
    ans2 = input("[You]: ")
    history.append(("Q2", ans2))

    # Generate final follow-up question (Q3)
    ques3 = chain2.invoke({"pre": ans2})
    print("\n[Bot]:", ques3)
    ans3 = input("[You]: ")
    history.append(("Q3", ans3))

# Summary
print("\n🎯 [Bot] Here's a summary of your responses:")
for i, (q, a) in enumerate(history, 1):
    print(f"Q{i}: {a}")


prompt3 = ChatPromptTemplate.from_messages([
    ("system","Recommend a career for the user based on the responses they gave"),
    ("user","these were the responses given by user {history} for the questions {ques1} {ques2} {ques3}")
])

chain3 = prompt3 | llm | parser

result = chain3.invoke({"history":history,
                       "ques1":ques1,
                       "ques2":ques2,
                       "ques3":ques3})
print(result)

Working and understanding of RunnableAssign

In [ ]:
## Automatic mapping by runnableAssign 

from langchain_core.runnables.passthrough import RunnableAssign


prompting = ChatPromptTemplate.from_template("Greet the user {name}")
result_chain = {'name' : lambda _: input('input')}| prompting|llm|StrOutputParser()
result_chain.invoke({})

In [ ]:
## Manual Mapping for clearner code 
from langchain_core.runnables.passthrough import RunnableAssign

assign = RunnableAssign({'name' : lambda _: input('input')})
prompting = ChatPromptTemplate.from_template("Greet the user {name}")
result_chain = assign | prompting|llm|StrOutputParser()
# Assigning the input to the name variable
result_chain.invoke({})

LLM - Document Summary Base 

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser

from langchain_nvidia_ai_endpoints import ChatNVIDIA

from pydantic import BaseModel, Field
from typing import List
from IPython.display import clear_output


class DocumentSummaryBase(BaseModel):
    running_summary: str = Field("", description="Running description of the document. Do not override; only update!")
    main_ideas: List[str] = Field([], description="Most important information from the document (max 3)")
    loose_ends: List[str] = Field([], description="Open questions that would be good to incorporate into summary, but that are yet unknown (max 3)")


summary_prompt = ChatPromptTemplate.from_template(
    "You are generating a running summary of the document. Make it readable by a technical user."
    " After this, the old knowledge base will be replaced by the new one. Make sure a reader can still understand everything."
    " Keep it short, but as dense and useful as possible! The information should flow from chunk to (loose ends or main ideas) to running_summary."
    " The updated knowledge base keep all of the information from running_summary here: {info_base}."
    "\n\n{format_instructions}. Follow the format precisely, including quotations and commas"
    "\n\nWithout losing any of the info, update the knowledge base with the following: {input}"
)